<a href="https://colab.research.google.com/github/oguchi-ebube/Sugar-Instance-Packager/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#import necessary libraries
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set(style="ticks",color_codes=True)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import gzip
import matplotlib.pyplot as plt

In [0]:
!cp "/content/drive/My Drive/Colab Notebooks/load_data.py" load_data.py
import load_data
!cp "/content/drive/My Drive/Colab Notebooks/data_helper_first.py" data_helper_first.py
import data_helper_first

In [0]:
def parse_gz(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def convert_to_DF(path):
    i = 0
    df = {}
    for d in parse_gz(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [8]:
# #insert path
path_df1 = '/content/drive/My Drive/Colab Notebooks/Products_review.csv'
path_df2 = '/content/drive/My Drive/Colab Notebooks/Products_reviews.csv'
path_df3 = '/content/drive/My Drive/Colab Notebooks/customer_review.csv'
path_df4 = '/content/drive/My Drive/Colab Notebooks/reviews_Sports_and_Outdoors_5.json.gz'


# Read the data in the CSV file using pandas
df1 = pd.read_csv(path_df1)
#print(df3)
df2 = pd.read_csv(path_df2)

df3 = pd.read_csv(path_df3, encoding = 'unicode_escape')
#print(df3)

!python load_data.py

df4 = convert_to_DF(path_df4)
# print(df4.columns)
df4 = df4.rename(index=str, columns={"reviewText": "review"})
df4 = df4.rename(index=str, columns={"overall": "rating"})
#print(df4.columns)

Data Succesfully Retrieved!


In [0]:
#concat the dataframes
df = pd.concat([df1, df2,df3,df4])
#df.drop(columns=['asin', 'asins', 'brand', 'categories', 'date', 'dateAdded','dateUpdated', 'helpful', 'imageURLs', 'keys', 'manufacturer','manufacturerNumber', 'name', 'primaryCategories',  'reviewTime', 'reviewerID', 'reviewerName', 'reviews.date','reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend','reviews.numHelpful', 'reviews.sourceURLs', 'reviews.title','reviews.username', 'sourceURLs', 'summary', 'title', 'unixReviewTime'])

# print(df[:5])
# print(df.columns)

df["label"] = np.where(df.rating >=3, 1, 0)



#print(df['label'])


#Number of the smaller class (negative reviews)
num_to_sample = len(df[df.label == 0])



df_neg = df[df["label"] == 0].sample(n=num_to_sample)


df_pos = df[df["label"] == 1].sample(n=num_to_sample)

In [0]:
df = pd.concat([df_neg, df_pos])


# Get the text from the dataframe
text = df["review"].values
#print(text)

# #clean strings
text= [data_helper_first.clean_str(str(sent)) for sent in text]



# Create labels from the dataframe
labels = df["label"].values


df = pd.concat([pd.DataFrame(text),pd.DataFrame(labels)],axis=1)
#print(len(df)) 144634
#print(df[3000:])


In [0]:
#print(text[:50])

In [0]:
X = text
y = labels

In [0]:
#change text lower cases and removal of white spaces
lower_text = []
for i in range(0,len(X)):
  s = str(X[i])
  s1 = s.strip()
  lower_text.append(s1.lower())
  #print(“After converting text to lower case\n\n”,lower_text)

In [0]:
#Remove punctuation
punc_text = []
for i in range(0,len(lower_text)):
  s2 = (lower_text[i])
  s3 = re.sub(r'[^\w\s2]',"",s2)
  punc_text.append(s3)
  #print(“After removed punctuation\n\n”,punc_text)

In [0]:
#Word vectorization
#Initialize the TF-IDF vectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2,max_df = 0.7,max_features =200000,norm='l2', encoding='latin-1',ngram_range=(3,3),stop_words='english')

In [16]:
#transform independent variable using TF-IDF vectorizer
print("\n")
X_tfidf = tfidf.fit_transform(punc_text)
print(X_tfidf.shape)
#print("After vectorized text data\n\n",X_tfidf)



(91132, 84887)


In [0]:
#Split the data into train and testing
X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state = 42)

In [0]:
# #Print training data
# print("\n")
# print("Training data\n\n",X_train,"\n",Y_train)
# print("\n\n")

In [0]:
# #Print testing data
# print("Testing data\n\n",X_test)
# print("\n\n")

In [0]:
#Build the SVM model
clf = LinearSVC()

In [21]:
#Fit train and test into the model
clf.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
#Predict the result
y_pred = clf.predict(X_test)

In [24]:
clf.score(X_test,Y_test)

0.8117410387710314

In [25]:
#classification report & confusion matrix
print("Confusion Matrix\n",confusion_matrix(Y_test,y_pred))
print("\n")
print("Classification Report\n",classification_report(Y_test,y_pred))
print("\n")
print("Accuracy : ",accuracy_score(Y_test,y_pred)*100)

Confusion Matrix
 [[ 9631  4108]
 [ 1039 12562]]


Classification Report
               precision    recall  f1-score   support

           0       0.90      0.70      0.79     13739
           1       0.75      0.92      0.83     13601

    accuracy                           0.81     27340
   macro avg       0.83      0.81      0.81     27340
weighted avg       0.83      0.81      0.81     27340



Accuracy :  81.17410387710315


In [0]:

import pickle

In [0]:

with open('/content/drive/My Drive/Colab Notebooks/training_models/svm_trained_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [0]:
# !pip install scikit-plot

In [0]:

# #Precision-recall graph
# plt.rcParams["figure.figsize"] = [16,10]
# import scikitplot as skplt
# probas = clf.predict(X_test)
# skplt.metrics.plot_precision_recall((Y_test,y_pred),probas)
# plt.title("Precision & Recall graph for SVM model")
# plt.show()

In [113]:

import numpy as np
label = {0:'negative', 1:'positive'}

example = ["i hate hate these make my feet hurt"]
Testing = tfidf.transform(example)
print(label[clf.predict(Testing)[0]])

positive
